In [1]:
from tensorflow.keras.layers import Flatten, Dense, Input, Lambda, Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf
import numpy as np

In [2]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/1512/PA/mhist_dataset.zip", 'r')
zip_ref.extractall("./")
zip_ref.close()

Mounted at /content/drive/


In [3]:
IMAGE_SIZE = (224, 224, 3)
NUM_CLASSES = 2
learning_rate_teacher_init = 1e-4
learning_rate_student_init = 1e-3
NUM_EPOCHS_init = 10
NUM_EPOCHS_ft = 25

# Load pre-trained ResNet50V2 and MobileNetV2 model

In [4]:
resnet_model = ResNet50V2(include_top=True, weights=None, input_shape=IMAGE_SIZE)
#freeze pre-trained resnet layers for initial epochs
for layer in resnet_model.layers:
    layer.trainable = False
    
#Design the last dense layer
x_teacher = Dense(2,activation=None)(resnet_model.output)
x_teacher.trainable = True

teacher_model_kd = Model(inputs=resnet_model.input, outputs=x_teacher)

mobile_model = MobileNetV2(include_top=True, weights=None, input_shape=IMAGE_SIZE)
#freeze pre-trained mobilenet layers for initial epochs
for layer in mobile_model.layers:
    layer.trainable = False
    
#Design the last dense layer
x_student = Dense(2,activation=None)(mobile_model.output)
x_student.trainable = True

student_model_kd = Model(inputs=mobile_model.input, outputs=x_student)

# Custom dataset and dataloader

In [5]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os
import math
from PIL import Image
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)
np.random.seed(42)
csv_path = './annotations.csv'
image_path = './images/images'
# model(torch.tensor(np.ones((1, 224,224,3))), training=True)
class Custom_Dataset(Dataset):
    def __init__(self, image_path, csv_path, transformer, split):
        self.image_path = image_path
        self.csv_path = csv_path
        annotations = pd.read_csv(csv_path)
        image_names = annotations ['Image Name']
        majority_vote_labels = annotations ['Majority Vote Label']
        self.train_image_names = np.array(image_names[annotations['Partition']=='train'])
        self.train_mv_labels = np.array(majority_vote_labels[annotations['Partition']=='train'])      
        self.test_image_names = np.array(image_names[annotations['Partition']=='test'])
        self.test_mv_labels = np.array(majority_vote_labels[annotations['Partition']=='test'])       
        self.transformer = transformer
        self.split = split
        
    def __len__(self):
        if self.split == 'train':
            return len(self.train_mv_labels)
        else:
            return len(self.test_mv_labels)
    
    def __getitem__(self, idx):
        if self.split == 'train':        
            image_full_name = os.path.join(self.image_path, self.train_image_names[idx])
            x = Image.open(image_full_name)
            if self.transformer is not None:
                x = self.transformer(x)
                x = x.permute(1,2,0)
            
            label = self.train_mv_labels[idx]
            if label == 'HP':
                y = np.array([1, 0])
            else:
                y = np.array([0, 1])
                
        else:        
            image_full_name = os.path.join(self.image_path, self.test_image_names[idx])
            x = Image.open(image_full_name)
            if self.transformer is not None:
                x = self.transformer(x)
                x = x.permute(1,2,0)

            label = self.test_mv_labels[idx]
            if label == 'HP':
                y = np.array([1, 0])
            else:
                y = np.array([0, 1])
                
        return x, y
    
custom_transformer = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(degrees=15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

custom_transformer_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
      
train_dataset = Custom_Dataset(image_path, csv_path, custom_transformer, 'train')
test_dataset = Custom_Dataset(image_path, csv_path, custom_transformer_test, 'test')
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(len(train_dataset))
print(len(test_dataset))

2175
977


# Teacher loss function

In [6]:
def compute_teacher_loss(images, labels):

    subclass_logits = teacher_model_kd(images, training=True)

    cross_entropy_loss_value = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          labels, subclass_logits))

    return cross_entropy_loss_value

# Student loss function

In [7]:
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 4 #temperature hyperparameter

def distillation_loss(teacher_logits, student_logits, temperature = DISTILLATION_TEMPERATURE):

    soft_targets = tf.nn.softmax(teacher_logits/temperature, axis = 1)
    
    return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss(images, labels):

    student_subclass_logits = student_model_kd(images, training=True)
    teacher_subclass_logits = teacher_model_kd(images, training=False)
    distillation_loss_value = distillation_loss(teacher_subclass_logits, student_subclass_logits, DISTILLATION_TEMPERATURE)

    cross_entropy_loss_value = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          labels, student_subclass_logits))
    
    total_loss = cross_entropy_loss_value * ALPHA + (1 - ALPHA) * distillation_loss_value

    return total_loss

# Train and evaluation

In [8]:
def tf_reduceat(data, at_array, axis=-1):
    split_data = tf.split(data, at_array, axis=axis)
    return tf.stack([tf.reduce_sum(i, axis=axis) for i in split_data], axis=axis)
    
# def compute_num_correct(model, images, labels):
#     class_logits = model(images, training=False)
#     if class_logits.shape[1] != NUM_CLASSES:
#         class_logits = tf_reduceat(class_logits, [8, 8])
        
#     return tf.reduce_sum(
#       tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
#               tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)

def compute_auc(model, images, labels):
    class_logits = model(images, training=False)
    if class_logits.shape[1] != NUM_CLASSES:
        class_logits = tf_reduceat(class_logits, [8, 8])
    
    scores = tf.nn.sigmoid(class_logits)
    m1 = tf.keras.metrics.AUC()
    m1.reset_state()
    m1.update_state(labels, scores)
    return m1.result().numpy()

def train_and_evaluate(model, compute_loss_fn, learning_rate, train_loader, test_loader, num_total):
    print('Start initial epochs')
    # do initial training epochs
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    for epoch in range(1, NUM_EPOCHS_init + 1):
        # Run training.
        print('Epoch {}: '.format(epoch), end='')
        for images, labels in train_loader:
            with tf.GradientTape() as tape:
                images = tf.convert_to_tensor(images.numpy())
                labels = tf.convert_to_tensor(labels.numpy())
                loss_value = compute_loss_fn(images, labels)

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

    #unfreezing all layers
    for layer in model.layers:
        layer.trainable = True
    print('\n')
    print('Start fine-tuning epochs')
    # do initial training epochs
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate/10)
    for epoch in range(1, NUM_EPOCHS_ft + 1):
        # Run training.
        print('Epoch {}: '.format(epoch), end='')
        for images, labels in train_loader:
            with tf.GradientTape() as tape:
                images = tf.convert_to_tensor(images.numpy())
                labels = tf.convert_to_tensor(labels.numpy())
                loss_value = compute_loss_fn(images, labels)

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    AUC_set = []
    for images, labels in test_loader:
        images = tf.convert_to_tensor(images.numpy())
        labels = tf.convert_to_tensor(labels.numpy())
        AUC_set.append(compute_auc(model, images, labels))

    test_AUC = sum(AUC_set)/len(AUC_set)
    print('\n')
    print("Test AUC: " + '{:.2f}'.format(test_AUC))
    return test_AUC


# Knowledge Distallation


In [9]:
print('Start training teacher model')
test_acc_teacher = train_and_evaluate(teacher_model_kd, compute_teacher_loss, learning_rate_teacher_init, train_loader, test_loader, len(test_dataset))
print('Test AUC for teacher model is {}'.format(test_acc_teacher))
print('-------------------------------------')
print('Start training student model')
test_acc_student = train_and_evaluate(student_model_kd, compute_student_loss, learning_rate_student_init, train_loader, test_loader, len(test_dataset))
print('Test AUC for student model is {}'.format(test_acc_student))

Start training teacher model
Start initial epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: 

Start fine-tuning epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: Epoch 11: Epoch 12: Epoch 13: Epoch 14: Epoch 15: Epoch 16: Epoch 17: Epoch 18: Epoch 19: Epoch 20: Epoch 21: Epoch 22: Epoch 23: Epoch 24: Epoch 25: 

Test AUC: 0.64
Test AUC for teacher model is 0.63623046875
-------------------------------------
Start training student model
Start initial epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: 

Start fine-tuning epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: Epoch 11: Epoch 12: Epoch 13: Epoch 14: Epoch 15: Epoch 16: Epoch 17: Epoch 18: Epoch 19: Epoch 20: Epoch 21: Epoch 22: Epoch 23: Epoch 24: Epoch 25: 

Test AUC: 0.62
Test AUC for student model is 0.6219758064516129


# Subclass Distallation


In [10]:
DISTILLATION_TEMPERATURE = 16

resnet_model = ResNet50V2(include_top=True, weights=None, input_shape=IMAGE_SIZE)
#freeze pre-trained resnet layers for initial epochs
for layer in resnet_model.layers:
    layer.trainable = False
    
#Design the last dense layer
x_teacher = Dense(16, activation=None)(resnet_model.output)
x_teacher.trainable = True

teacher_model_SbD = Model(inputs=resnet_model.input, outputs=x_teacher)

mobile_model = MobileNetV2(include_top=True, weights=None, input_shape=IMAGE_SIZE)
#freeze pre-trained mobilenet layers for initial epochs
for layer in mobile_model.layers:
    layer.trainable = False
    
#Design the last dense layer
x_student = Dense(16, activation=None)(mobile_model.output)
x_student.trainable = True

student_model_SbD = Model(inputs=mobile_model.input, outputs=x_student)

def compute_teacher_loss_SbD(images, labels):     
        
    subclass_logits_Z = teacher_model_SbD(images, training=True)
    subclass_logits_P = tf.nn.softmax(subclass_logits_Z)
    subclass_logits_P_sumed = tf_reduceat(subclass_logits_P, [8, 8])
    
    cee = tf.keras.losses.CategoricalCrossentropy()
    cross_entropy_loss_value = cee(labels, subclass_logits_P_sumed)
    temperature = 120
    BETA = 0.1
    
    def Auxiliary_loss(subclass_logits_Z, temperature):
        mean = tf.math.reduce_mean(subclass_logits_Z, axis = 1)
        var = tf.math.reduce_variance(subclass_logits_Z, axis = 1)
        normalizer = tf.keras.layers.Normalization(axis=0, mean=mean, variance=var)
        subclass_logits_Z = normalizer(subclass_logits_Z)

        subclass_logits_Z_transposed = tf.transpose(subclass_logits_Z)
        logics_vectors = tf.matmul(subclass_logits_Z, subclass_logits_Z_transposed)
        logics_vectors /= temperature
        exp_values = tf.math.exp(logics_vectors)
        log_term = tf.reduce_sum(exp_values, 1)
        log_results = tf.math.log(log_term)
        log_results -= (1/temperature)
        log_results -= math.log(batch_size)
        auxiliary_loss = tf.reduce_mean(log_results)
        return auxiliary_loss
    
    auxiliary_loss = Auxiliary_loss(subclass_logits_Z, temperature)
    teacher_loss = cross_entropy_loss_value + BETA * auxiliary_loss
    return teacher_loss

def distillation_loss_SbD(teacher_logits, student_logits, temperature = DISTILLATION_TEMPERATURE):

    soft_targets = tf.nn.softmax(teacher_logits/temperature, axis = 1)
    
    return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss_SbD(images, labels):

    student_subclass_logits_Z = student_model_SbD(images, training=True)

    teacher_subclass_logits_Z = teacher_model_SbD(images, training=False)
    
    student_subclass_logits_P = tf.nn.softmax(student_subclass_logits_Z)
    student_subclass_logits_P_sumed = tf_reduceat(student_subclass_logits_P, [8, 8])
    
    cee = tf.keras.losses.CategoricalCrossentropy()
    cross_entropy_loss_value = cee(labels, student_subclass_logits_P_sumed)
    
    distillation_loss_value = distillation_loss_SbD(teacher_subclass_logits_Z, student_subclass_logits_Z, DISTILLATION_TEMPERATURE)

    student_loss = cross_entropy_loss_value * ALPHA + (1 - ALPHA) * distillation_loss_value

    return student_loss

print('Start training teacher model')
test_acc_teacher = train_and_evaluate(teacher_model_SbD, compute_teacher_loss_SbD, learning_rate_teacher_init, train_loader, test_loader, len(test_dataset))
print('Test AUC for teacher model is {}'.format(test_acc_teacher))
print('-------------------------------------')
print('Start training student model')
test_acc_student = train_and_evaluate(student_model_SbD, compute_student_loss_SbD, learning_rate_student_init, train_loader, test_loader, len(test_dataset))
print('Test AUC for student model is {}'.format(test_acc_student))

Start training teacher model
Start initial epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: 

Start fine-tuning epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: Epoch 11: Epoch 12: Epoch 13: Epoch 14: Epoch 15: Epoch 16: Epoch 17: Epoch 18: Epoch 19: Epoch 20: Epoch 21: Epoch 22: Epoch 23: Epoch 24: Epoch 25: 

Test AUC: 0.52
Test AUC for teacher model is 0.5161290322580645
-------------------------------------
Start training student model
Start initial epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: 

Start fine-tuning epochs
Epoch 1: Epoch 2: Epoch 3: Epoch 4: Epoch 5: Epoch 6: Epoch 7: Epoch 8: Epoch 9: Epoch 10: Epoch 11: Epoch 12: Epoch 13: Epoch 14: Epoch 15: Epoch 16: Epoch 17: Epoch 18: Epoch 19: Epoch 20: Epoch 21: Epoch 22: Epoch 23: Epoch 24: Epoch 25: 

Test AUC: 0.62
Test AUC for student model is 0.6219758064516129
